In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr'

%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('/home/mink/notebooks/CameraTraps')  # append this repo to PYTHONPATH
sys.path.append('/home/mink/lib/ai4eutils')

In [4]:
import json
import os
from collections import Counter, defaultdict
from random import sample
from shutil import copyfile
from multiprocessing.pool import ThreadPool

import numpy as np
import pandas as pd
from tqdm import tqdm

import path_utils  # ai4eutils

from data_management.megadb.schema import sequences_schema_check
from data_management.megadb.converters.cct_to_megadb import process_sequences

# islandconservation_rodents_2020

In [5]:
dataset_name = 'islandconservation_rodents_2020'

container_root = '/mink_disk_0/camtraps/island-conservation-private/'  
path_prefix = 'rodents-2021.01.05/contrapest/Rodent Photos/'

path_to_output = f'/home/mink/camtraps/data/megadb_jsons/{dataset_name}.json' 
path_to_output_temp = f'/home/mink/camtraps/data/megadb_jsons/{dataset_name}_temp.json' 

## Step 0 - Add an entry to the `datasets` table

Done

## Step 1 - Prepare the `sequence` objects to insert into the database

Extracted camera (~location) from file name. There is a spreadsheet in the folder called ContraPestAug2020TimelapseTemplate.xlsx with timestamp and MD/reviewer labels.

In [6]:
folder = os.path.join(container_root, path_prefix)

paths = path_utils.recursive_file_list(folder)
len(paths)
paths = sorted([p.split(folder)[1] for p in paths if path_utils.is_image_file(p)])
len(paths)

853

845

In [7]:
paths[0]

'Aug2020/CAM003.100_BTCF.IMG_0071.JPG'

In [9]:
locations = set()
sequences = []

for p in paths:
    location = p.split('.')[0]
    locations.add(location)
    sequences.append({
        'dataset': dataset_name,
        'seq_id': f'dummy_{len(sequences)}',
        'images': [
            {
                'file': p,
                'frame_num': 1 # only one image, but easier for ingesting the annotations
            }
        ],
        'location': location,
        'class': ['rodent']
    })

In [10]:
len(locations)

36

In [11]:
len(sequences)
sequences[700]

845

{'dataset': 'islandconservation_rodents_2020',
 'seq_id': 'dummy_700',
 'images': [{'file': 'Oct2020/cam060.100_BTCF.IMG_0496.JPG', 'frame_num': 1}],
 'location': 'Oct2020/cam060',
 'class': ['rodent']}

## Step 2 - Pass the schema check

Once your metadata are in the MegaDB format for `sequence` items, we check that they conform to the format's schema.

If the format conforms, the following messages will be printed:

```
Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.
```

For large datasets, the second step will take some time (~ a minute). 

Otherwise there will be an error message describing what's wrong. Please fix the issues until all checks are passed. You might need to write some snippets of code to loop through the `sequence` items to understand which entries have problems.

In [12]:
%%time

sequences_schema_check.sequences_schema_check(sequences)

Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.
CPU times: user 115 ms, sys: 0 ns, total: 115 ms
Wall time: 114 ms


In [13]:
with open(path_to_output_temp, 'w', encoding='utf-8') as f:
    json.dump(sequences, f, indent=1, ensure_ascii=False)

### Step 2b - copy images to flat folder

In [14]:
def copy_file(src_path, dst_path):
    return copyfile(src_path, dst_path)

In [15]:
%%time

path_pairs = []
for seq in tqdm(sequences):
    seq_id = seq['seq_id']
    for im in seq['images']:
        src_path = os.path.join(container_root, path_prefix, im['file'])
        assert os.path.exists(src_path), src_path
        frame = 1
        dst_path = os.path.join('/mink_disk_0/camtraps/imerit12b', 
                                f'{dataset_name}.seq{seq_id}.frame{frame}.jpg')
        path_pairs.append((src_path, dst_path))

100%|██████████| 845/845 [00:00<00:00, 1115.30it/s]

CPU times: user 37.9 ms, sys: 17.5 ms, total: 55.4 ms
Wall time: 760 ms


In [16]:
len(path_pairs)
path_pairs[-100]

845

('/mink_disk_0/camtraps/island-conservation-private/rodents-2021.01.05/contrapest/Rodent Photos/Sep2020/cam051.100_BTCF.IMG_0102.JPG',
 '/mink_disk_0/camtraps/imerit12b/islandconservation_rodents_2020.seqdummy_745.frame1.jpg')

In [17]:
%%time

with ThreadPool(8) as pool:
    dst_paths = pool.starmap(copy_file, path_pairs)

CPU times: user 725 ms, sys: 2.02 s, total: 2.74 s
Wall time: 3.41 s


In [18]:
len(dst_paths)

845